Creating a op property which gives from what operation the current value object is being created.

In [529]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [528]:
class Value:
    def __init__(self,data,_children=(),_op = '',label = ''):
        self.data = data
        self._prev = _children
        self._op = _op
        self.grad = 0.0
        self.label = label
        self._backpropUnit = lambda: None
    
    def __repr__(self):
        return f"Value(data= {self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out =  Value(self.data+other.data,(self,other),'+')
        def _backpropUnit(): # assuming out.grad was set before calling the function here
            self.grad+= 1.0*out.grad
            other.grad+= 1.0*out.grad
        out._backpropUnit = _backpropUnit
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out =  Value(self.data*other.data,(self,other),'*')
        def _backpropUnit(): # assuming out.grad was set before calling the function here
            self.grad +=other.data*out.grad
            other.grad+= self.data*out.grad
        out._backpropUnit = _backpropUnit
        return out

    def __truediv__(self,other):
        return self*other**-1

    def __rmul__(self,other): #they are in reverse order
        return self*other
    
    def __radd__(self,other):
        return self+other
    
    def __neg__(self):
        return self*-1 

    def __sub__(self,other):
        return self+(-other)
    
    def __rsub__(self,other):
        # print(f"self: {self} other: {other}")
        return -self+other

    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t,(self,),_op = 'tanh')
        def _backpropUnit(): # assuming out.grad was set before calling the function here
            # print(f"out.grad : {out.grad}")
            self.grad += out.grad*(1-t**2)
        out._backpropUnit = _backpropUnit
        return out
    
    def exp(self):
        x = math.exp(self.data)
        out = Value(x,(self,),_op=f'exp({self.data})',label='exp')
    
        def _backpropUnit():
            self.grad+= out.grad*out.data
        out._backpropUnit = _backpropUnit
    
        return out
    
    def __pow__(self,other):
        assert isinstance(other,(int,float)) , "only supporting int and float as power"
        out = Value(self.data**other,(self,),f'**{other}',f'**{other}')
        def _backpropUnit():
            self.grad+=other * (self.data ** (other - 1)) * out.grad
        out._backpropUnit = _backpropUnit
        return out

    def topologicalOrder(self):
        topo = []
        visited = set()
    
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        return topo

    def printForwardPass(self):
        topo = self.topologicalOrder()
        for v in topo:
            print(f"{v.label} : {v.grad} ")

    def backward(self):
        topo = self.topologicalOrder()
        self.clean()
        self.grad = 1.0
        for node in reversed(topo):
            node._backpropUnit()


    def clean(self):
        nodes = set()
        def _clean(v):
            if v not in nodes:
                nodes.add(v) # ensures atmax only one level deep the repititions will be.
                v.grad = 0
                for child in v._prev:
                    _clean(child)
        _clean(self)

    


Now here the values are good but in order to be able to perform backpropagation we need a connecting tissue... and it should be such that if c = a+b then c should be able to point to a and b. 
empty tuple as childrens and while initialising we will use set.... but lets for now use list... 

Writing the logic of backpropagation

In [515]:
#inputs x1,x2
x1 = Value(2.0,label='x1')
x2 = Value(-3.0,label='x2')
#weights
w1,w2 = Value(-4.0,label="w1"),Value(5.0,label='w2')
#biases
b = Value(4.5,label='b')
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1+x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2+b; n.label = 'n'
o = n.tanh(); o.label = 'o'



In [654]:


class Neuron:
    def __init__(self,nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self, x):
        act = sum((xi*wi for xi,wi in zip(x,self.w)),self.b)
        out = act.tanh()
        return out

class Layer:
    def __init__(self,nin,nop):
        self.neurons = [Neuron(nin) for _ in range(nop)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs if len(outs)>1 else outs[0]

class MLP:
    def __init__(self,nin,nops):
        sz = [nin]+nops
        self.layers =[Layer(sz[i],sz[i+1]) for i in range(len(nops))]
    
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x) # x is updated as the output of the layer(x) which is then fed to another layer.
        return x

x = [2.0,3.0,8]
n = MLP(3,[4,4,1])
n(x)


Value(data= 0.7068876976283466)